<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/LinearProgramming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving linear problems in Python

This notebook compares different alternatives to solve optimization problems in Python

We solve a standard linear programming problem of the form

```
min  c*x
s.t. A*x <= b
     x >= 0
```

## Import basic packages

In [16]:
# Import 
import os
import numpy as np
import pandas as pd

## Create random vectors and matrix of linear problem

In [43]:
# Import
import random
# Number of variables and constraints
nvar = 10
ncon = 5
# Random vectors and matrix of linear problem
c = [round(abs(random.gauss(0,1)),2) for i in range(nvar)]
A = [[round(random.gauss(0,1),2) for i in range(nvar)] for j in range(ncon)]
b = [round(random.gauss(0,1),2) for j in range(ncon)]

## Solve linear problem using scipy

In [29]:
# Import
from scipy.optimize import linprog
# Solve problem
res = linprog(c, A_ub=A, b_ub=b)
# Print results
print(res)

     con: array([], dtype=float64)
     fun: 0.7654054054074841
 message: 'Optimization terminated successfully.'
     nit: 7
   slack: array([2.26594595e+00, 3.72797297e+00, 5.32907052e-13, 8.74594595e-01,
       3.36216216e-01])
  status: 0
 success: True
       x: array([5.85579428e-13, 2.39189189e+00, 1.86099448e-13, 5.97607163e-13,
       5.23603797e-13, 1.33538571e-13, 8.57285335e-14, 2.12200317e-13,
       1.34743901e-13, 1.32345031e-11])


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


## Solve linear problem using pulp

In [33]:
# Install
!pip install pulp
# Import
import pulp as pu
# Model
m = pu.LpProblem('LinearProblem',pu.LpMinimize)
# Variables
x = pu.LpVariable.dicts('x', list(range(nvar)), lowBound=0, cat="Continuous")
# Objective function
m += sum(c[i]*x[i] for i in range(nvar))
# Constraints
for j in range(ncon):
  m += sum(A[j][i]*x[i] for i in range(nvar)) <= b[j]
# Solve problem
m.solve()
# Princ results
print('Status:', pu.LpStatus[m.status])
print('Optimal value:',pu.value(m.objective))
print('Optimal solution:',[x[i].varValue for i in range(nvar)])

Status: Optimal
Optimal value: 0.765405408
Optimal solution: [0.0, 2.3918919, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## Solve linear problem using Pyomo

In [44]:
# Install
!pip install pyomo
# Import
import pyomo.environ as pe
# Model
m = pe.ConcreteModel()
# Sets
m.i = pe.Set(initialize=range(nvar),ordered=True)
m.j = pe.Set(initialize=range(ncon),ordered=True)
# Variables
m.z = pe.Var()
m.x = pe.Var(m.i,within=pe.NonNegativeReals)
# Objective function
def obj_rule(m):
  return sum(c[i]*m.x[i] for i in m.i)
m.obj = pe.Objective(rule=obj_rule)
# Constraints
def con_rule(m,j):
  return sum(A[j][i]*m.x[i] for i in m.i) <= b[j]
m.con = pe.Constraint(m.j,rule=con_rule)
# Solve problem using NEOS server
os.environ['NEOS_EMAIL'] = 'xxx@gmail.com'
res = pe.SolverManagerFactory('neos').solve(m,opt=pe.SolverFactory('cplex'))
print(res['Solver'][0])
# Print results
print('Optimal value:',m.obj())
print('Optimal solution:',[m.x[i].value for i in m.i])


Status: ok
Message: CPLEX 12.10.0.0\x3a optimal solution; objective 1.965689993159713; 4 dual simplex iterations (0 in phase I)
Termination condition: optimal
Id: 0

Optimal value: 1.965689993159713
Optimal solution: [0.0, 0.20698960609953482, 1.8609711540065732, 0.24795208461936313, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
